In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_validate
import tqdm
import random
import seaborn as sns

from common import generate_multi_signal, get_data, get_freq_data, signal_cyclic_shift

In [2]:
X_origin, y_origin = get_data('./data/na62_11_pulses.txt', is_one_signal=False)
X_one_signal, y_one_signal = get_data('./data/na62_11_pulses.txt', is_one_signal=True)
ARGMIN_DISTR = np.argmin(X_one_signal, axis=1)



def prepare_data(X_origin, y_origin, tau_range, alpha_range, data_size=1000, to_print=False,\
                                                                         start_point=70, end_point=950):
    pos_size = int(data_size/2)
    neg_size = data_size - pos_size
    
    X = []
    y = []
    for i in range(data_size):
        alpha = random.choice(alpha_range)
        tau = random.choice(tau_range)
        
        if i < pos_size:
#             X.append(random.choice(X_one_signal))
            X.append(generate_multi_signal(X_origin, y_origin, tau, alpha,\
                                           argmin_distr=ARGMIN_DISTR)['first_impulse'][start_point:end_point])

            y.append(1)
        else:
            X.append(generate_multi_signal(X_origin, y_origin, tau, alpha,\
                                           argmin_distr=ARGMIN_DISTR)['multi_impulse'][start_point:end_point])
            y.append(0)
   
    X = np.array(X)
    y = np.array(y)
   
    if to_print:
#         print("X positive shape:", X_positive.shape)
#         print("y positive shape:", y_positive.shape)
#         print("X negative shape:", X_negative.shape)
#         print("y negative shape:", y_negative.shape)
        print("X shape:", X.shape)
        print("y shape:", y.shape)
    
    X, y = shuffle(X, y)

    return X, y

In [3]:
class Spacal_data(Dataset):
    def __init__(self, tX, tY = None, train=True):         
        if train == True:
            self.X = torch.Tensor(tX).type(torch.LongTensor)
            self.y = torch.Tensor(tY).type(torch.LongTensor)
            self.train = True
            
        else:
            self.X = torch.Tensor(tX).type(torch.LongTensor)
            self.y = None
            self.train = False
            
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        #preprocessing on demand trades computations for memory
        if self.train == True:
            return self.X[idx], self.y[idx] 
        else:
            return self.X[idx]

In [4]:
alpha_range = np.array([np.around(10**i, decimals=4) for i in np.arange(-3, 3.1, 0.1)])
# alpha_range = np.array([2])
tau_range = np.arange(-25, 25, 1)
# alpha_range = np.arange(1, 10, 1)

X, y = prepare_data(X_origin, y_origin,tau_range, alpha_range, data_size=5000, to_print=True)

X shape: (5000, 880)
y shape: (5000,)


In [5]:
TEST_SIZE = len(X)//10

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE)

In [6]:
train_loader = Spacal_data(X_train, y_train)
test_loader = Spacal_data(X_test, y_test)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [8]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [9]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
BATCH_SIZE = 128
TEST_BATCH_SIZE = 1024
LR = 0.0001
TO_USE_MOMENTUM = 1
EPOCHS_NUM = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# train_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=True, download=True,
#                    transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=BATCH_SIZE, shuffle=True)

# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=False, transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=TEST_BATCH_SIZE, shuffle=True)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=TO_USE_MOMENTUM)


for epoch in range(1, EPOCHS_NUM + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [20, 1, 5, 5], but got 1-dimensional input of size [880] instead